query for efi2 deficit for bmi, this table will be used for bmi in descriptive analysis also so have coded to cateogrise bmi for everyone and identify missing bmi 


In [2]:
library(ggplot2)
library(bigrquery)
library(tidyverse)
library(ggplot2)
library(lubridate)
library(stats)

ProjectId = "yhcr-prd-bradfor-bia-core"

targetdb2 <- 'yhcr-prd-bradfor-bia-core.CB_FDM_PrimaryCare'
targetdb2 <-gsub(' ','',targetdb2)

targetdb3 <- 'yhcr-prd-bradfor-bia-core.CB_2172'
targetdb3 <- gsub(' ','',targetdb3)

targetdb4 <- 'yhcr-prd-bradfor-bia-core.ref_Lookups'
targetdb4 <- gsub(' ','',targetdb4)


In [ ]:
sql_bmi <- paste0('REATE OR REPLACE TABLE `yhcr-prd-bradfor-bia-core.CB_2172.temp_cb_2172_efi2_bmi` as (
#bring all snomed and ctv3codes for care home cohort in primary into the cte
with a as (
  select
  person_id,
  snomedcode,
  ctv3code,
  dateevent,
  cast(numericvalue as bignumeric) as bmi
  from `yhcr-prd-bradfor-bia-core.CB_FDM_PrimaryCare.tbl_srcode`
  where person_id in(select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`)),
  #join care home entry date from msater table
  a_master as (
    select
    a.*,
    j.first_episodestartdate
    from a
    left join `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` j on
    a.person_id = j.person_id),
#filter for bmi snomed codeswithout numericvalues
  b as (
    select
    person_id,
    snomedcode,
    dateevent,
    bmi
    from a_master
    where snomedcode in(select SNOMEDCT_CONCEPTID from `yhcr-prd-bradfor-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where deficit = 'Obesity' AND Otherinstructions IS NULL) AND dateevent < first_episodestartdate),
#filter for bmi with xctv3 codes 
  c as (
    select
    person_id,
    ctv3code, 
    dateevent,
    bmi
    from a_master
    where ctv3code in(select CTV3 from `yhcr-prd-bradfor-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where deficit = 'Obesity' AND Otherinstructions IS NOT NULL) AND dateevent < first_episodestartdate),
#union distinct individuals identified from ctv3 and snomed ct codes together
  snomed_ctv3 as (
    select
    * from b

    union distinct

    select *
    from c),
#apply dummy variable
bmi_seq as (
    select
    person_id,
    dateevent,
    row_number() over(partition by person_id order by dateevent desc) as code_sequence,
    bmi
    from snomed_ctv3),

bmi_known as (
  select
  person_id,
  dateevent,
  bmi
  from bmi_seq
  where code_sequence = 1)

  select
  j.person_id,
  bmi,
  bmi_known.dateevent,
  case when bmi <18.5 then 'underweight'
  when bmi between 18.5 and 24.9999999999 then 'normal'
  when bmi between 25.0 and 29.9999999999 then 'overweight'
  when bmi >= 30.0 then 'obese'
  else 'missing'
  end as bmi_factor
  from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` j
  left join bmi_known on
  j.person_id = bmi_known.person_id 
  order by person_id)
;
',sep="")

tbl_bmi <-bq_project_query(ProjectId,sql_bmi)
bmi_data <- bq_table_download(tbl_bmi)
